# Reference 
* Reference to [DeepLearningProject's github](https://github.com/bjpublic/DeepLearningProject/tree/main/04_%EC%9E%91%EB%AC%BC_%EC%9E%8E_%EC%82%AC%EC%A7%84_%EC%A7%88%EB%B3%91_%EB%B6%84%EB%A5%98)
* Reference to [pytorch.org - Transfer Learning](https://tutorials.pytorch.kr/beginner/transfer_learning_tutorial.html)

# 학습 목표 
* Image Classification (잎 사진의 종류와 질병 유무 분류)
* Transfer Learning에 대한 개념 확립 
* 기본 CNN 모델의 학습 방식과 어떤 차이점이 있는지 숙지하기 

# 1. Prepare Dataset 


## 1-1. Dataset download 
* 원본 데이터: https://data.mendeley.com/datasets/tywbtsjrjv/1
* 수정된 데이터: [bjpublic's github](https://github.com/bjpublic/DeepLearningProject/tree/main/04_%EC%9E%91%EB%AC%BC_%EC%9E%8E_%EC%82%AC%EC%A7%84_%EC%A7%88%EB%B3%91_%EB%B6%84%EB%A5%98)

## 1-2. Dataset split 
* 데이터를 train, val, test 디렉토리로 분할
* 디렉토리 생성 방법; [pathlib.Path.mkdir](https://stackoverflow.com/questions/273192/how-can-i-safely-create-a-nested-directory)

### (1) 데이터 분할을 위한 디렉토리 생성 

In [1]:
import os 
import os.path as osp 
from pathlib import Path

In [2]:
dpath = r'./dataset'
class_list = os.listdir(dpath) 
print(f"{len(class_list)} classes", end="\n\n")   # 분류 클래스 확인 
print(f"{class_list}")   # 분류 클래스 확인 

33 classes

['Strawberry___healthy', 'Potato___Early_blight', 'Corn___Common_rust', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Apple___Cedar_apple_rust', 'Apple___Apple_scab', 'Tomato___healthy', 'Potato___healthy', 'Corn___Northern_Leaf_Blight', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Cherry___Powdery_mildew', 'Peach___healthy', 'Apple___healthy', 'Apple___Black_rot', 'Cherry___healthy', 'Peach___Bacterial_spot', 'Tomato___Tomato_mosaic_virus', 'Strawberry___Leaf_scorch', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Grape___Black_rot', 'Potato___Late_blight', 'Tomato___Leaf_Mold', 'Pepper,_bell___healthy', 'Tomato___Bacterial_spot', 'Corn___healthy', 'Tomato___Early_blight', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Tomato___Late_blight', 'Tomato___Septoria_leaf_spot', 'Tomato___Target_Spot', 'Pepper,_bell___Bacterial_spot']


In [3]:
base_dir = r'./processed'

for split in ['train', 'val', 'test']: 
    for clss in class_list:
        # train, val, test 용 디렉토리 생성 및 
        # 각 split 폴더 하위에 클래스 목록 디렉토리 생성 
        Path(osp.join(base_dir, split, clss)).mkdir(parents=True, exist_ok=True) 

### (2) 데이터 분할과 클래스별 데이터 개수 확인 

In [4]:
import random 
import math 
import shutil

In [5]:
def data_copy(img_frames:list, ori_path:str, dst_path:str): 
    for fname in img_frames: 
        src = osp.join(ori_path, fname) # 복사할 원본 파일의 경로 
        dst = osp.join(dst_path, fname) # 복사 후 저장할 경로  
        shutil.copyfile(src, dst) # src -> dst 경로로 복사 

In [6]:
for clss in class_list:
    ori_path = osp.join(dpath, clss) # 원본 데이터 경로 
    fnames = os.listdir(ori_path) # 해당 디렉토리에 포함된 파일 목록을 가져옴 
#    print(f"num_data of {clss}: {len(fnames)}")

    # split each class data into 6:2:2
    train_size = math.floor(len(fnames)*0.6)
    val_size = math.floor(len(fnames)*0.2)
    test_size = math.floor(len(fnames)*0.2)

    train_fnames = fnames[:train_size]
    val_fnames = fnames[train_size:(train_size + val_size)] 
    test_fnames = fnames[(train_size + val_size):
                        (train_size + val_size + test_size)]  

    # data copy 
    train_dst = osp.join(base_dir, 'train', clss)
    data_copy(train_fnames, ori_path, train_dst)

    val_dst = osp.join(base_dir, 'val', clss)
    data_copy(val_fnames, ori_path, val_dst)

    test_dst = osp.join(base_dir, 'test', clss)
    data_copy(test_fnames, ori_path, test_dst)

### (3) DataLoader 

In [7]:
from torch.utils.data import DataLoader

import torchvision.transforms as T 
from torchvision.datasets import ImageFolder

In [8]:
BATCH_SIZE = 256 

transform_base = T.Compose([
    T.Resize((64,64)),
    T.ToTensor(), # 이미지를 Tensor 형태로 변환 
    ])

In [9]:
train_dataset = ImageFolder(root=r"./processed/train", transform=transform_base)
val_dataset = ImageFolder(root=r"./processed/val", transform=transform_base)
test_dataset = ImageFolder(root=r"./processed/test", transform=transform_base)


print(train_dataset.classes)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [10]:
train_loader = DataLoader(  train_dataset, 
                            batch_size=BATCH_SIZE, 
                            shuffle=True, 
                            num_workers=4 )

val_loader = DataLoader(val_dataset, 
                        batch_size=BATCH_SIZE, 
                        shuffle=True, 
                        num_workers=4 )                            

In [11]:
imgs, labels = next(iter(train_loader)) 
print(imgs.size(), labels.size())
print(imgs.device)

torch.Size([256, 3, 64, 64]) torch.Size([256])
cpu


# 2. Baseline blocks 

In [12]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(device)

cuda


## 2-1. CNN model 

In [14]:
class base_Net(nn.Module): 
    def __init__(self, num_classes:int=33): 
        super(base_Net, self).__init__() # nn.Module 내에 있는 메소드를 상속받기 위함 
        
        # backbone 
        # -------- 
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(2,2),
            nn.Dropout(p=0.25)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3,stride=1, padding=1), 
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(2,2),
            nn.Dropout(p=0.25)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3,stride=1, padding=1), 
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(2,2),
            nn.Dropout(p=0.25)
        )

        # building classifier
        self.classifier = nn.Sequential(
            nn.Linear(4096, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, num_classes)
            )

        # init. weights 
        self._initialize_weights()

    def forward(self, x:torch.Tensor): 
        assert x.ndimension() == 4, f"input tensor should be 4D, but got {x.ndimension()}D"

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = x.view(-1, 64*8*8) # (B,64,8,8) -> (B, 4096)        
        x = self.classifier(x)

        return x 
    
    def _initialize_weights(self): 
        for m in self.modules(): 
            if isinstance(m, nn.Conv2d): 
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels # 파라미터 개수 
                m.weight.data.normal_(0,  math.sqrt(2. / n))
                if m.bias is not None: 
                    m.bias.data.zero_() 
            elif isinstance(m, nn.BatchNorm2d):                     
                m.weight.data.fill_(1)
                m.weight.data.zero_()
            elif isinstance(m, nn.Linear):                                     
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [15]:
model = base_Net(num_classes= len(class_list)).to(device)

x = torch.randn(4, 3, 64, 64).to(device)

logit = model(x)
print(logit.size())

torch.Size([4, 33])


## 2-2. train loop 

In [16]:
def train(model, train_loader, optimizer, loss_fn): 
    model.train() 
    train_loss = 0.0 
    train_correct = 0

    for batch_idx, (data, labels) in enumerate(train_loader): 
        data, labels = data.to(device), labels.to(device)

        # forward 
        # --------
        logit = model(data) 
        _, argmax_idx = torch.max(logit, dim=1) 
        loss = loss_fn(logit, labels) 
        
        # backward & step  
        # ---------------
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 

## 2-3. evaluation loop 

In [17]:
def evaluate(model, dataloader, loss_fn): 
    model.eval() 
    total_loss = 0.0 
    total_correct = 0

    with torch.no_grad(): 
        for batch_idx, (data, labels) in enumerate(dataloader): 
            data, labels = data.to(device), labels.to(device)

            # forward 
            # --------
            logit = model(data)
            _, argmax_idx = torch.max(logit, dim=1) 
            loss = loss_fn(logit, labels) 

            # === accumulate batch loss & accuracy 
            total_loss += loss.item()
            total_correct += torch.sum(argmax_idx == labels.data).item()

        avg_loss = total_loss / len(dataloader.dataset) 
        acc = 100. * total_correct / len(dataloader.dataset) 
    
    return avg_loss, acc

# 3. Run model training 

In [18]:
import time 
import copy 

import torch.optim as optim 
from torch.optim.lr_scheduler import StepLR

In [19]:
loss_fn = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=1e-4) 

# learning rate scheduler. 
exp_lr_scheduler = StepLR(  optimizer, # optim object 
                            step_size=7,
                            gamma=0.1
                        )

In [20]:
EPOCHS = 25 

best_acc = 0.0 
best_model_wts = copy.deepcopy(model.state_dict()) 

for epoch in range(1, EPOCHS+1): 
    since = time.time() 

    # Train 
    # ---------
    train(model, train_loader, optimizer, loss_fn)
    train_loss, train_acc = evaluate(model, train_loader, loss_fn)

                                                                
    # Schedule the learning rate for next the epoch of training.
    # ---------------------------------------------------------
    exp_lr_scheduler.step()  

    # Validation 
    # ------------
    val_loss, val_acc = evaluate(model, val_loader, loss_fn)


    # New best 
    # -------------
    if val_acc > best_acc: 
        best_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())

        # save the model parameters 
        torch.save(best_model_wts, "./new_baseline_best.pth")

    # Log 
    # ---------- 
    time_elapsed = time.time() - since 
    print(f'-------------- epoch {epoch} ----------------')
    print(f'train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%')   
    print(f'val Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%')
    print(f'Completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s') 

-------------- epoch 1 ----------------
train Loss: 0.0126, Accuracy: 15.73%
val Loss: 0.0129, Accuracy: 15.83%
Completed in 0m 19s
-------------- epoch 2 ----------------
train Loss: 0.0097, Accuracy: 29.98%
val Loss: 0.0099, Accuracy: 30.23%
Completed in 0m 20s
-------------- epoch 3 ----------------
train Loss: 0.0083, Accuracy: 38.43%
val Loss: 0.0085, Accuracy: 38.34%
Completed in 0m 20s
-------------- epoch 4 ----------------
train Loss: 0.0075, Accuracy: 44.70%
val Loss: 0.0077, Accuracy: 44.97%
Completed in 0m 20s
-------------- epoch 5 ----------------
train Loss: 0.0067, Accuracy: 50.89%
val Loss: 0.0069, Accuracy: 50.63%
Completed in 0m 20s
-------------- epoch 6 ----------------
train Loss: 0.0063, Accuracy: 52.69%
val Loss: 0.0065, Accuracy: 52.77%
Completed in 0m 19s
-------------- epoch 7 ----------------
train Loss: 0.0058, Accuracy: 56.04%
val Loss: 0.0060, Accuracy: 56.33%
Completed in 0m 20s
-------------- epoch 8 ----------------
train Loss: 0.0058, Accuracy: 56.22%

***
# 4. Transfer Learning 
* ResNet50 모델을 불러온 후, '작물 잎을 분류하는 task'에 맞춰서 Fine-Tuning 하기

## 4-1. DataLoader 

In [20]:
data_transforms = {
    'train': T.Compose([T.Resize([64,64]), 
        T.RandomHorizontalFlip(p=0.5), T.RandomVerticalFlip(p=0.5),  
        T.RandomCrop(52), T.ToTensor(), 
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]),
    
    'val': T.Compose([T.Resize([64,64]),  
        T.RandomCrop(52), T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

In [21]:
data_dir = r"./processed"

image_datasets = {x: ImageFolder(root=osp.join(data_dir, x), transform=data_transforms[x]) 
                                                                for x in ['train', 'val']} 
dataloaders = {x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 
                                                                        for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}          


class_names = image_datasets['train'].classes
print(class_names)

['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


## 4-2. Get Pre-trained Model

In [22]:
from torchvision import models 

In [23]:
resnet = models.resnet50(pretrained=True)

print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## 4-3. Model Freezing 
* learnable parameters 가 업데이트 되지 않도록 ```requires_grad``` 를 OFF 한다. 
* ```children()``` 메소드는 자식 모듈을 반복 가능한 객체로 반환한다. 
    * ```resnet.children()``` 은 생성한 resnet 모델의 모든 Layer 정보를 담고 있음. 
    * 아래 freeze_model() 함수는 ResNet50에 존재하는 10개의 Layer 중에서 1~5번 Layer의 파라미터만 업데이트되도록 고정함 

In [32]:
def freeze_model(model): 
    ct = 0 
    for child in model.children():  
        ct += 1 
        if ct < 6: 
            for parameter in child.parameters(): 
                parameter.requires_grad = False 

In [33]:
freeze_model(resnet)

## 4-4. Replacing the top of the model (for fine-tune)
* ```backbone``` : bottom 
* ```classifier``` : top 

In [34]:
print(resnet.fc)

Linear(in_features=2048, out_features=33, bias=True)


In [35]:
torch.manual_seed(11) 

resnet.fc = nn.Linear(2048, len(class_names))

## 4-5. Training 
* ```filter(lambda p: p.requires_grad, resnet.parameters()``` 
    * 모델의 일부 Layer의 파라미터만 업데이트하기 위해서 ```requires_grad=True```로 설정된 Layer의 파라미터만 반환 

In [36]:
resnet = resnet.to(device)

loss_fn = nn.CrossEntropyLoss() 
optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=1e-4) 

# learning rate scheduler. 
exp_lr_scheduler = StepLR(  optimizer, # optim object 
                            step_size=7,
                            gamma=0.1
                        )

In [37]:
EPOCHS = 25 

best_acc = 0.0 
best_model_wts = copy.deepcopy(resnet.state_dict()) 

for epoch in range(1, EPOCHS+1): 
    since = time.time() 

    # Train 
    # ---------
    train(resnet , train_loader, optimizer, loss_fn)
    train_loss, train_acc = evaluate(resnet , train_loader, loss_fn)

                                                                
    # Schedule the learning rate for next the epoch of training.
    # ---------------------------------------------------------
    exp_lr_scheduler.step()  
    lr = [x['lr'] for x in optimizer.param_groups]

    # Validation 
    # ------------
    val_loss, val_acc = evaluate(resnet , val_loader, loss_fn)


    # New best 
    # -------------
    if val_acc > best_acc: 
        best_acc = val_acc
        best_model_wts = copy.deepcopy(resnet.state_dict())

        # save the model parameters 
        torch.save(best_model_wts, "./new_resnet_best.pth")

    # Log 
    # ---------- 
    time_elapsed = time.time() - since 
    print(f'-------------- epoch {epoch} ----------------')
    print(f"learning rate: {lr}")
    print(f'train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%')   
    print(f'val Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%')
    print(f"Best val Acc: {best_acc:.4f}")
    print(f'Completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s') 

-------------- epoch 1 ----------------
learning rate: [0.0001]
train Loss: 0.0093, Accuracy: 42.84%
val Loss: 0.0096, Accuracy: 42.37%
Best val Acc: 42.3708
Completed in 0m 21s
-------------- epoch 2 ----------------
learning rate: [0.0001]
train Loss: 0.0070, Accuracy: 60.43%
val Loss: 0.0073, Accuracy: 58.87%
Best val Acc: 58.8684
Completed in 0m 20s
-------------- epoch 3 ----------------
learning rate: [0.0001]
train Loss: 0.0058, Accuracy: 67.39%
val Loss: 0.0061, Accuracy: 65.53%
Best val Acc: 65.5276
Completed in 0m 20s
-------------- epoch 4 ----------------
learning rate: [0.0001]
train Loss: 0.0051, Accuracy: 70.88%
val Loss: 0.0054, Accuracy: 68.68%
Best val Acc: 68.6819
Completed in 0m 19s
-------------- epoch 5 ----------------
learning rate: [0.0001]
train Loss: 0.0045, Accuracy: 73.22%
val Loss: 0.0049, Accuracy: 70.72%
Best val Acc: 70.7222
Completed in 0m 19s
-------------- epoch 6 ----------------
learning rate: [0.0001]
train Loss: 0.0042, Accuracy: 75.11%
val Loss:

***
# 5. 모델 평가 

In [38]:
def inference_test(model, dataloader): 
    model.eval() 
    total_correct = 0

    with torch.no_grad(): 
        for batch_idx, (data, labels) in enumerate(dataloader): 
            data, labels = data.to(device), labels.to(device)

            # forward 
            # --------
            logit = model(data)
            _, argmax_idx = torch.max(logit, dim=1) 
            
            # === accumulate accuracy 
            total_correct += torch.sum(argmax_idx == labels.data).item()

        acc = 100. * total_correct / len(dataloader.dataset) 
    
    return acc

## 5-1. Baseline CNN 모델 평가

In [32]:
transform_base = T.Compose([T.Resize([64,64]), T.ToTensor()])
test_base = ImageFolder(root='./processed/test',transform=transform_base)  
test_loader_base = DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [39]:
baseline = torch.load('new_baseline_best.pth') 

custom_model = base_Net(num_classes= len(class_list)).to(device)
custom_model.load_state_dict(baseline)
custom_model = custom_model.to(device)


test_accuracy = inference_test(custom_model, test_loader_base)

print(f'baseline test acc: {test_accuracy:.4f}% ')

baseline test acc: 56.8657% 


## 5-2. Transfer Learning 모델 평가

In [34]:
transform_resNet = T.Compose([
        T.Resize([64,64]),  
        T.RandomCrop(52), T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])
    
test_resNet = ImageFolder(root='./processed/test', transform=transform_resNet) 
test_loader_resNet = DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [38]:
# prepare model 
resnet_test = models.resnet50(pretrained=False)
resnet_test.fc = nn.Linear(2048, len(class_names))

# load state_dict 
resnet_states = torch.load('new_resnet_best.pth') 

resnet_test.load_state_dict(resnet_states)
resnet_test = resnet_test.to(device) 


# inference 
resnet_accuracy = inference_test(resnet_test , test_loader_base)
print(f'resnet test acc: {resnet_accuracy:.4f}%')

resnet test acc: 73.6012%
